In [ ]:
from pyspark.sql.types import *
from datetime import datetime

## Step 1: Create `feature_releases_roadmap` Table

In [ ]:
print("🔄 Creating table: feature_releases_roadmap")
print("=" * 70)

schema_roadmap = StructType([
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("feature_description", StringType(), True),
    StructField("workload", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("release_date", TimestampType(), True),  # Nullable for planned features
    StructField("release_type", StringType(), True),
    StructField("release_status", StringType(), True),
    StructField("is_preview", BooleanType(), False),
    StructField("is_planned", BooleanType(), False),
    StructField("is_shipped", BooleanType(), False),
    StructField("last_modified", TimestampType(), False),
    StructField("source_url", StringType(), True),
    StructField("source", StringType(), True),
    StructField("extracted_date", TimestampType(), False)
])

df_roadmap = spark.createDataFrame([], schema_roadmap)
table_path = "Tables/feature_releases_roadmap"
df_roadmap.write.format("delta").mode("overwrite").save(table_path)

print("✅ Table created: feature_releases_roadmap")
print(f"   Schema: {len(schema_roadmap.fields)} columns")
print("\n   💡 Includes planned/future features and historical tracking")

## Step 2: Create `preview_features_active` Table

In [ ]:
print("\n🔄 Creating table: preview_features_active")
print("=" * 70)

schema_preview_active = StructType([
    StructField("setting_name", StringType(), False),
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("similarity_score", DoubleType(), False),
    StructField("is_enabled", BooleanType(), False),
    StructField("delegate_to_tenant", BooleanType(), True),
    StructField("detected_date", TimestampType(), False),
    StructField("release_date", TimestampType(), True),
    StructField("release_status", StringType(), True),
    StructField("source_url", StringType(), True),
    StructField("days_since_release", IntegerType(), True)
])

df_preview_active = spark.createDataFrame([], schema_preview_active)
table_path = "Tables/preview_features_active"
df_preview_active.write.format("delta").mode("overwrite").save(table_path)

print("✅ Table created: preview_features_active")
print(f"   Schema: {len(schema_preview_active.fields)} columns")

## Step 3: Create `feature_alerts` Table

In [ ]:
print("\n🔄 Creating table: feature_alerts")
print("=" * 70)

schema_alerts = StructType([
    StructField("alert_id", StringType(), False),
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("alert_type", StringType(), False),
    StructField("severity", StringType(), False),
    StructField("message", StringType(), False),
    StructField("setting_name", StringType(), True),
    StructField("similarity_score", DoubleType(), True),
    StructField("days_since_release", IntegerType(), True),
    StructField("alert_date", TimestampType(), False),
    StructField("acknowledged", BooleanType(), False),
    StructField("acknowledged_date", TimestampType(), True),
    StructField("acknowledged_by", StringType(), True)
])

df_alerts = spark.createDataFrame([], schema_alerts)
table_path = "Tables/feature_alerts"
df_alerts.write.format("delta").mode("overwrite").save(table_path)

print("✅ Table created: feature_alerts")
print(f"   Schema: {len(schema_alerts.fields)} columns")

## Step 4: Create Helper SQL Views

In [ ]:
print("\n🔄 Creating helper SQL views...")
print("=" * 70)

# View 1: Roadmap Upcoming Features
spark.sql("""
    CREATE OR REPLACE VIEW vw_roadmap_upcoming AS
    SELECT 
        feature_name,
        feature_description,
        product_name,
        workload,
        release_type,
        release_status,
        release_date,
        is_preview,
        is_planned,
        last_modified,
        CASE 
            WHEN release_date IS NULL THEN NULL
            ELSE DATEDIFF(release_date, CURRENT_DATE())
        END as days_until_release
    FROM feature_releases_roadmap
    WHERE is_planned = true
      AND (release_date IS NULL OR release_date >= CURRENT_DATE())
    ORDER BY release_date ASC NULLS LAST, last_modified DESC
""")
print("✅ vw_roadmap_upcoming - Planned/upcoming features")

# View 2: Active Preview Features
spark.sql("""
    CREATE OR REPLACE VIEW vw_active_preview_features AS
    SELECT 
        feature_name,
        workload,
        setting_name,
        days_since_release,
        similarity_score,
        release_date,
        detected_date,
        is_enabled
    FROM preview_features_active
    WHERE is_enabled = true
    ORDER BY detected_date DESC
""")
print("✅ vw_active_preview_features - Currently enabled previews")

# View 3: Critical Alerts
spark.sql("""
    CREATE OR REPLACE VIEW vw_critical_alerts AS
    SELECT 
        alert_id,
        feature_name,
        workload,
        alert_type,
        severity,
        message,
        alert_date,
        acknowledged
    FROM feature_alerts
    WHERE acknowledged = false 
      AND severity IN ('Critical', 'Warning')
    ORDER BY 
        CASE severity 
            WHEN 'Critical' THEN 1 
            WHEN 'Warning' THEN 2 
            ELSE 3 
        END,
        alert_date DESC
""")
print("✅ vw_critical_alerts - Unacknowledged critical/warning alerts")

# View 4: Feature Release Timeline
spark.sql("""
    CREATE OR REPLACE VIEW vw_feature_timeline AS
    SELECT 
        feature_name,
        product_name,
        workload,
        release_type,
        release_status,
        is_preview,
        is_planned,
        is_shipped,
        release_date,
        CASE 
            WHEN release_date IS NULL THEN NULL
            ELSE DATEDIFF(CURRENT_DATE(), release_date)
        END as days_since_release,
        last_modified
    FROM feature_releases_roadmap
    ORDER BY release_date DESC NULLS LAST
""")
print("✅ vw_feature_timeline - Complete release timeline")

print("\n✅ All SQL views created successfully")

## ✅ Setup Complete!

In [ ]:
print("\n" + "=" * 70)
print("🎉 FEATURE TRACKING SETUP COMPLETED!")
print("=" * 70)

# Verify tables
tables = ["feature_releases_roadmap", "preview_features_active", "feature_alerts"]
print("\n📋 Tables created:")
for table in tables:
    try:
        count = spark.read.format("delta").load(f"Tables/{table}").count()
        print(f"  ✅ {table}: {count} rows")
    except Exception as e:
        print(f"  ❌ {table}: ERROR - {e}")

# Verify views
views = ["vw_roadmap_upcoming", "vw_active_preview_features", "vw_critical_alerts", "vw_feature_timeline"]
print("\n📋 Views created:")
for view in views:
    try:
        spark.sql(f"SELECT * FROM {view} LIMIT 1")
        print(f"  ✅ {view}")
    except Exception as e:
        print(f"  ❌ {view}: ERROR - {e}")

print("\n" + "=" * 70)
print("📚 Next Step:")
print("=" * 70)
print("\n  → Run 'Load_Feature_Tracking' notebook to populate the tables")
print("\n💡 Schedule Load_Feature_Tracking to run daily for continuous monitoring")
print("\n" + "=" * 70)